In [1]:
# load modules
import netCDF4 as nc
import glob
import numpy as np
import datetime
import os

In [2]:
# system timezone to UTC
# everything will be done in UTC
# UTC = CET-1h/CEST-2h
os.environ['TZ'] = 'UTC'

In [3]:
# date of interest
# need to find correct file path
doi = '2021-06-20' # year-month-day

In [4]:
# list the to file(s) at the doi
y_oi = doi.split('-')[0]
m_oi = doi.split('-')[1]
d_oi = doi.split('-')[2]

fnames = glob.glob('/Volumes/Elements/data/msevi_rss/raw_tb/'+y_oi+'/'+m_oi+'/'+d_oi+'/*')
fnames.sort()

In [5]:
# for loop -> loop over different files
# looping over all files is available in the .py script (msevi_nat_to_netcdf.py)
# in this notebook: i = [from 0 to number of files]
i = 0

In [9]:
# get the i-th filename 
fname = fnames[i]
ftext = fnames[i]

In [10]:
# get date and time information of the file
ftext = ftext.split("/")[9]
ftext = ftext.split(".")[1]
ftext = ftext.split("-")[1]

year = list(ftext)[0]+list(ftext)[1]+list(ftext)[2]+list(ftext)[3]
month = list(ftext)[4]+list(ftext)[5]
day = list(ftext)[6]+list(ftext)[7]
hour = list(ftext)[8]+list(ftext)[9]
minute = list(ftext)[10]+list(ftext)[11]
second = list(ftext)[12]+list(ftext)[13]

In [11]:
# open netCDF file
ds = nc.Dataset(fname)

In [12]:
# get IR_108 channel Brightness Temperature
tb = ds.variables['IR_108'][:]

In [15]:
# transpose the IR_108 array
tb = tb[np.arange(tb.shape[0]-1,0-1,-1),:]

In [18]:
# put array into a big Brightness Temperature dataset

# if i = 0 in the for loop:
# https://stackoverflow.com/questions/53882600/numpy-append-multiple-different-np-arrays-in-one-axis-of-existing-np-array
Tb = np.ma.asanyarray([tb]*len(fnames))

# if i >= 1 in the for loop:
Tb[i] = tb

In [20]:
############
# SENSING TIME AS UNIX-TIMESTAMP
# seconds since 1970-01-01 00:00:00 (UTC)
dt_str = year+'-'+month+'-'+day+' '+hour+':'+minute+':'+second
dt_object = datetime.datetime.fromisoformat(dt_str)
timestamp = datetime.datetime.timestamp(dt_object)

In [21]:
# put array into a big time array

# if i = 0 in the for loop:
times = np.ma.array([int(timestamp)]*len(fnames))

# if i >=1 in the for loop:
times[i] = timestamp

In [296]:
## !! END OF FOR LOOP HERE !!

In [74]:
############
# GET X,Y DIMENSIONS
# define latitude and longitude array
lat = ds.variables['latitude'][:]
lon = ds.variables['longitude'][:]
#lat_dim = np.ma.masked_array(np.arange(lat.min(),lat.max()+0.001,(lat.max()-lat.min())/(lat.shape[0]-1)))
#lon_dim = np.ma.masked_array(np.arange(lon.min(),lon.max()+0.001,(lon.max()-lon.min())/(lon.shape[1]-1)))
lat_dim = lat[:,0][::-1]
lon_dim = lon[0]

In [75]:
############
# CREATE THE NETCDF FILE
nc_fn = '/Volumes/Elements/data/msevi_rss/tobac_tb/'+y_oi+'/'+m_oi+'/'+'OLR_'+year+month+day+'.nc'
nc_ds = nc.Dataset(nc_fn, 'w', format = 'NETCDF4')

In [76]:
# define dimensions
time = nc_ds.createDimension('time', len(times))
lat = nc_ds.createDimension('lat', len(lat_dim))
lon = nc_ds.createDimension('lon', len(lon_dim))

In [77]:
# define variables
tb = nc_ds.createVariable('tb', 'f4', ('time', 'lat', 'lon',))
time = nc_ds.createVariable('time', 'f4', ('time',))
lat = nc_ds.createVariable('lat', 'f4', ('lat',))
lon = nc_ds.createVariable('lon', 'f4', ('lon',))

In [78]:
# define units
tb.units = 'K'
time.units = 'seconds since 1970-1-1'
lat.units = 'degrees_north'
lon.units = 'degrees_east'

In [79]:
# define other names
tb.long_name = 'Brightness Temperature'
time.standard_name = 'time'
lat.standard_name = 'latitude'
lon.standard_name = 'longitude'

In [80]:
# define axis
time.axis = 'T'
lat.axis = 'Y'
lon.axis = 'X'

In [81]:
# define others
time.calendar = 'gregorian'
lat.datum = 'WGS84'
lon.datum = 'WGS84'

In [82]:
# define spacing
lat.spacing = str((lat_dim[len(lat_dim)-1]-lat_dim[0])/(len(lat_dim)-1))
lon.spacing = str((lon_dim[len(lon_dim)-1]-lon_dim[0])/(len(lon_dim)-1))

In [83]:
# assign data to variables
lat[:] = lat_dim
lon[:] = lon_dim
time[:] = times
tb[:] = Tb

In [84]:
# close the netCDF
nc_ds.close()